# <center> 🧽 Data Cleaning </center>

## Table of Contents

- [1. Introduction](#1-introduction)
- [2. Loading Data](#2-loading-data)

## 1. Introduction
[Back to Table of Contents](#table-of-contents)

---

| ⚡ Description: Introducing the project ⚡ |
| :--------------------------- |

---

<a id="two"></a>
## 2. Loading Data
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data in our environment ⚡ |
| :--------------------------- |

---

### Import packages

In [ ]:
# Import packages
import re

import numpy as np
import pandas as pd

### Rental Properties

In [ ]:
# Load the rental listings dataset
rental_listings = pd.read_csv("../data/raw/rental_listings_for_capital_cities.csv")

# Display the first few rows of the dataset and it's shape
display("Rentals", rental_listings.head(), rental_listings.shape)

In [ ]:
# Print a summary of the dataset
rental_listings.info()

In [ ]:
# Print the total number of null values in the dataset
rental_null_values = rental_listings.isnull().sum().sum()
print(f"There are {rental_null_values} null values in the rental dataset")

In [ ]:
# Print the number of null values in each column
rental_listings.isnull().sum()

In [ ]:
rental_listings[rental_listings.isnull().sum(axis=1) >= 4]

In [ ]:
rental_listings.drop(
    rental_listings[rental_listings.isnull().sum(axis=1) >= 2].index, inplace=True)

In [ ]:
# Get the total number of duplicate rows in the dataset
rental_duplicates = rental_listings[rental_listings.duplicated()]
num_rental_duplicates = len(rental_duplicates)
print(f"There are {num_rental_duplicates} duplicates in the rental dataset ")

In [ ]:
# Drop duplicate rows and keep the first occurrence
rental_listings = rental_listings.drop_duplicates().reset_index(drop=True)

In [ ]:
# Remove any information following the text 'Points of Interest' in the 'property_features' column
rental_listings["property_features"] = rental_listings["property_features"].str.replace(
    r'Points of Interest.*', '', regex=True
)

rental_listings["property_features"] = rental_listings["property_features"].str.strip()

In [ ]:
patterns = {
    'Listing Number': r'Listing Number(\d+)',
    'Type of Property': r'Type of Property\s*(House|Apartment / Flat|Townhouse|Commercial|Industrial|Vacant Land|Farm)',
    #'Description': r'Description([A-Za-z]+)',
    #'Lifestyle': r'Lifestyle([A-Za-z]+)',
    #'Occupation Date': r'Occupation Date([\d\w\s]+?)Deposit Requirements',
    #'Deposit Amount': r'Deposit Amount:\s*R\s?([\d,]+)',
    'Pets Allowed': r'Pets Allowed(Yes|No)',
    'Furnished': r'Furnished(Yes|No)',
    'Bedrooms': r'Bedrooms?\s*(\d+)',
    'Bathrooms': r'Bathrooms?\s*(\d+)',
    'Kitchen': r'Kitchens?\s*(\d+)',
    'Lounge': r'Lounges?\s*(\d+)',
    'Dining Room': r'Dining Room(\d+)',
    'Parking': r'Parking(\d+)',
    #'Special Features': r'Special Features(\d+)',
    #'Backup Water': r'Backup Water(Yes|No)',
    'Erf Size (m²)': r'Erf Size\s*(\d+)\s*m²',
    'Floor Area (m²)': r'Floor Area\s*(\d+)\s*m²',
    'Garden': r'Gardens?\s*(\d+)',
    'Pool': r'Pools?\s*(\d+)',
    'Garage': r'Garages?\s*(\d+)'
}



# Function to extract features from text
def extract_features(text):
    return {
        field: (match := re.search(pattern, text)
                ) and match.group(1).strip() or None for field, pattern in patterns.items()
    }
    
# Apply extraction row-wise
extracted_df = rental_listings['property_features'].apply(extract_features).apply(pd.Series)

# Combine original df with the extracted columns
results_df = pd.concat([rental_listings, extracted_df], axis=1)

# View result
print(results_df)

In [ ]:
def extract_bedrooms_from_property_title(title):
    if pd.isna(title):
        return None
    match = re.search(r'(\d+)[-\s]?bedroom', title.lower())
    return int(match.group(1)) if match else None

results_df['Bedrooms'] = results_df['Bedrooms'].fillna(
    results_df.apply(lambda row: extract_bedrooms_from_property_title(row['property_title']) if pd.isna(row['Bedrooms']) else row['Bedrooms'], axis=1))

# Optional: convert to integer if needed
results_df['Bedrooms'] = pd.to_numeric(results_df['Bedrooms'], errors='coerce').astype('Int64')

In [ ]:
results_df["Bedrooms"].isnull().sum()

In [ ]:
# Normalize spaces
results_df['price'] = results_df['price'].str.replace(
    r'\s+', '', regex=True)

# Extract price amount
results_df['price (in rands)'] = results_df['price'].str.extract(
    r'R?(\d+)', expand=False)
results_df['price (in rands)'] = pd.to_numeric(results_df['price (in rands)'], errors='coerce')

# Now safely extract price type from the actual price string
results_df['price_type'] = results_df['price'].str.extract(
    r'(PerMonth|m2|m²|Perm²|PerDay)', expand=False
)

# Standardize price type to make it more readable
results_df['price_type'] = results_df['price_type'].replace({
    'PerMonth': 'Per Month',
    'm2': 'Per m²',
    'Perm²': 'Per m²',
    'PerDay': 'Per Day'
})

# Print out the first five rows of the table to see the results
results_df.head()

In [ ]:
# Get the number of rows in the dataset in the 'price' column that are equal to 'POA'
poa_mask = results_df['price'].astype(
    str).str.contains("POA", case=False, na=False)
num_poa = poa_mask.sum()

# Print the total number of rows
print(f"There are {num_poa} listings with 'POA' in the price column.")

In [ ]:
poa_rows = results_df[results_df['price'].astype(
    str).str.lower() == 'POA']
print(poa_rows)

In [ ]:
results_df['price_type'].value_counts()

In [ ]:
# Extract the location from the property_title column
rental_listings['suburb'] = rental_listings['property_title'].str.extract(
    r'to Rent in (.+)', expand=False)

# Remove the trailing numbers from the suburb column
rental_listings['suburb'] = (
    rental_listings['suburb'].str.replace(r'-\d+$', '', regex=True).str.strip())

# Print the first five rows of the table to see the results
rental_listings.head()

In [ ]:
rental_listings['suburb'].isnull().sum()

In [ ]:
city_province = pd.read_csv("../data/raw/south_africa_provinces_capitals.csv")
city_list = city_province['city'].dropna().str.strip().unique()

In [ ]:
def extract_city_from_location(location, city_list):
    if pd.isna(location):
        return None
    
    
    location_lower = location.lower()
    for city in city_list:
        if city.lower() in location_lower:
            return city
    return None

In [ ]:
# Apply the extract_city_from_location function to extract the city
rental_listings['city'] = rental_listings['location'].apply(
    lambda loc: extract_city_from_location(loc, city_list))

In [ ]:
rental_listings['city'].value_counts()

In [ ]:
rental_listings['city'].isnull().sum()

In [ ]:
# Create a mapping of suburb to kmown cities
suburb_to_city_map = rental_listings[rental_listings['city'].notna()].drop_duplicates(subset=['suburb', 'city']).set_index('suburb')['city'].to_dict()

# Fill missing city values based on the suburb
rental_listings['city'] = rental_listings.apply(lambda row: suburb_to_city_map.get(row['suburb'], row['city']) if pd.isna(row['city']) else row['city'], axis= 1)   

In [ ]:
rental_listings['city'].isnull().sum()

In [ ]:
# Drop the city rows with null values
rental_listings = rental_listings.dropna(subset=['city'])

In [ ]:
# Merge the city with its matching province
rental_listings = rental_listings.merge(city_province, on='city', how='left')

In [ ]:
rental_listings.isnull().sum()

In [ ]:
# Suburbs 
rental_listings['suburb'].value_counts()

In [ ]:
rental_listings.info()

In [ ]:
rental_listings.head(10)

In [ ]:
results_df['Erf Size (m²)'].value_counts()

In [ ]:
results_df.head(20)

In [ ]:
results_df['Parking'].value_counts()

In [ ]:
# Set display options
pd.set_option('display.max_columns', None)        # Show all columns
# Show all rows (if you're looking at multiple)
pd.set_option('display.max_rows', None)
# Show full content in each cell
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)              # Prevent line wrapping

In [ ]:
results_df["Parking"] = pd.to_numeric(results_df["Parking"], errors='coerce')
results_df[results_df["Parking"] == 3].sort_values("Parking")

In [ ]:
results_df['Garage'].value_counts()

In [ ]:
results_df['Bedrooms'].isnull().sum()

In [ ]:
results_df['Bedrooms'].isnull().sum()

In [ ]:
results_df.info()

In [ ]:
results_df.head(20)

In [ ]:
results_df['Furnished'].value_counts()

In [ ]:
results_df['Type of Property'].value_counts()

### Listings for sale

In [ ]:
listings_for_sale = pd.read_csv("../data/raw/listings_for_sale_in_capital_cities.csv")
display("Listings for sale", listings_for_sale.head(), listings_for_sale.shape)

In [ ]:
# Get information about the listings_for_sale document
listings_for_sale.info()

In [ ]:
# See the number of null values in each column
listings_for_sale.isnull().sum()

In [ ]:
# Drop listings where the property_title is null
listings_for_sale = listings_for_sale.dropna(subset=['property_title']).reset_index(drop=True)

In [ ]:
listings_for_sale_duplicates = listings_for_sale[listings_for_sale.duplicated()]
num_listings_for_sale_duplicates = len(listings_for_sale_duplicates)
print(f"There are {num_listings_for_sale_duplicates} duplicates in the rental dataset ")

In [ ]:
# Drop duplicate rows and keep the first occurrence
listings_for_sale = listings_for_sale.drop_duplicates().reset_index(drop=True)

In [ ]:
print(listings_for_sale['price'].str.contains(
    'POA', case=False, na=False).sum())

In [ ]:
"""_summary_
# Remove rows where 'price' contains 'POA' (case-insensitive)
listings_for_sale = listings_for_sale[~listings_for_sale['price'].astype(
    str).str.contains('POA', case=False, na=False)]
rental_listings.reset_index(drop=True, inplace=True)
    """

In [ ]:
listings_for_sale['price_cleaned'] = listings_for_sale['price'].str.extract(
    r'(R\s?\d[\d\s]*)')

listings_for_sale['price_cleaned'] = listings_for_sale['price_cleaned'].str.replace(
    r'\s+', '', regex=True)

listings_for_sale['price (in rands)'] = listings_for_sale['price_cleaned'].str.extract(
    r'R?(\d+)', expand=False)

# Print out the first five rows to see the results
listings_for_sale.head()

In [ ]:
# Extract the location from the listing_slug column
listings_for_sale['suburb'] = listings_for_sale['property_title'].str.extract(
    r'for Sale in (.+)', expand=False)

# Remove the trailing numbers from the suburb column
listings_for_sale['suburb'] = (
    listings_for_sale['suburb'].str.replace(r'-\d+$', '', regex=True).str.strip())

# Print the first five rows of the table to see the results
listings_for_sale.head()

In [ ]:
# Apply the extract_city_from_location function to extract the function
listings_for_sale['city'] = listings_for_sale['location'].apply(
    lambda loc: extract_city_from_location(loc, city_list))

In [ ]:
listings_for_sale['city'].isnull().sum()

In [ ]:
# Create a mapping of suburb to kmown cities
suburb_to_city_map = listings_for_sale[listings_for_sale['city'].notna()].drop_duplicates(
    subset=['suburb', 'city']).set_index('suburb')['city'].to_dict()

# Fill missing city values based on the suburb
listings_for_sale['city'] = listings_for_sale.apply(lambda row: suburb_to_city_map.get(
    row['suburb'], row['city']) if pd.isna(row['city']) else row['city'], axis=1)

In [ ]:
# See the number of unique values in the city column
listings_for_sale['city'].value_counts()

In [ ]:
listings_for_sale.isnull().sum()

In [ ]:
# Drop the rows where the city_column is empty
listings_for_sale = listings_for_sale.dropna(subset=['city'])

In [ ]:
listings_for_sale.isnull().sum()

In [ ]:
# Merge to get the matching province for the listings
listings_for_sale = listings_for_sale.merge(city_province, on='city', how='left')

In [ ]:
listings_for_sale.isnull().sum()

In [ ]:
poa_mask = listings_for_sale['price'].astype(
    str).str.contains("POA", case=False, na=False)
num_poa = poa_mask.sum()

print(f"There are {num_poa} listings with 'POA' in the price column.")